In [1]:
import sys
sys.path.append('../')
import pandas as pd
from src.preprocess import load_and_preprocess_data

master_df = load_and_preprocess_data('../data')

✅ Data loading and preprocessing complete.


In [5]:
# --- Corrected Data Preparation for Optimization ---

# First, let's find our top 5 products by revenue
top_products = master_df.groupby('product_id')['price'].sum().nlargest(5).index

# Filter our master_df to only include these top products
top_products_df = master_df[master_df['product_id'].isin(top_products)]

# ---- Correctly calculate the number of weeks in the dataset ----
min_date = master_df['order_purchase_timestamp'].min()
max_date = master_df['order_purchase_timestamp'].max()
number_of_weeks = (max_date - min_date).days / 7

# Now, create a simple summary table for our optimization problem
# We'll get the average price and the CORRECT weekly demand forecast
product_summary = top_products_df.groupby('product_id').agg(
    avg_price=('price', 'mean'),
    # Correctly calculate weekly demand: total units sold / number of weeks
    weekly_demand_forecast=('order_item_id', lambda x: x.count() / number_of_weeks)
).reset_index()

# ---- Business Assumptions (same as before) ----
promo_discount = 0.10  # 10%
demand_lift = 1.2      # 20% increase

product_summary['discount_price'] = product_summary['avg_price'] * (1 - promo_discount)
product_summary['promo_demand_forecast'] = product_summary['weekly_demand_forecast'] * demand_lift

print("--- Corrected Data for Optimization ---")
print(product_summary)

--- Corrected Data for Optimization ---
                         product_id    avg_price  weekly_demand_forecast  \
0  6cdd53843498f92890544667809f1595   350.834615                1.500000   
1  99a4788cb24856965c36a24e339b6058    88.167131                4.692308   
2  bb50f2e236e5eea0100680137654686c   327.615385                1.875000   
3  d1c427060a0f73f6b889a5c7c61f2ac4   137.651633                3.298077   
4  d6160fb7873f184099d9bc95e30376af  1397.124000                0.336538   

   discount_price  promo_demand_forecast  
0      315.751154               1.800000  
1       79.350418               5.630769  
2      294.853846               2.250000  
3      123.886469               3.957692  
4     1257.411600               0.403846  


In [3]:
pip install pulp

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pulp

# --- 1. Define Business Constraints ---
# Let's set a total weekly budget for discounts.
# The discount for a promoted item is (avg_price - discount_price).
weekly_discount_budget = 500.00

# --- 2. Initialize the Optimization Model ---
# We define our model and state that our goal is to maximize.
model = pulp.LpProblem("Promotion_Optimization", pulp.LpMaximize)

# --- 3. Define Decision Variables ---
# We need to decide FOR EACH product, "should we promote it or not?"
# This is a binary decision (0 = No, 1 = Yes).
# We create a dictionary of variables, one for each product_id.
products = product_summary['product_id'].tolist()
promo_vars = pulp.LpVariable.dicts("Promote", products, cat='Binary')

# --- 4. Define the Objective Function ---
# Our goal is to MAXIMIZE total revenue.
# Total Revenue = Revenue from non-promoted items + Revenue from promoted items
revenue_expression = pulp.lpSum([
    # Revenue if NOT promoted (promo_vars[p] will be 0)
    (1 - promo_vars[p]) * product_summary.loc[i, 'avg_price'] * product_summary.loc[i, 'weekly_demand_forecast'] +
    # Revenue if PROMOTED (promo_vars[p] will be 1)
    (promo_vars[p]) * product_summary.loc[i, 'discount_price'] * product_summary.loc[i, 'promo_demand_forecast']
    for i, p in enumerate(products)
])
model += revenue_expression, "Total_Revenue"

# --- 5. Define the Budget Constraint ---
# The total cost of discounts cannot exceed our budget.
# Total Discount Cost = Sum of (discount_per_item * forecasted_promo_demand * promo_decision)
discount_cost_expression = pulp.lpSum([
    promo_vars[p] * (product_summary.loc[i, 'avg_price'] - product_summary.loc[i, 'discount_price']) * product_summary.loc[i, 'promo_demand_forecast']
    for i, p in enumerate(products)
])
model += discount_cost_expression <= weekly_discount_budget, "Total_Discount_Budget_Constraint"

# --- 6. Solve the Problem ---
print("Solving the optimization problem...")
model.solve()
print("✅ Solution found.")

# --- 7. Display the Results ---
print("\n--- Optimal Promotion Plan ---")
total_revenue = 0
total_discount_cost = 0

for i, p in enumerate(products):
    if promo_vars[p].varValue == 1:
        print(f"✅ Promote Product: {p}")
        revenue = product_summary.loc[i, 'discount_price'] * product_summary.loc[i, 'promo_demand_forecast']
        discount_cost = (product_summary.loc[i, 'avg_price'] - product_summary.loc[i, 'discount_price']) * product_summary.loc[i, 'promo_demand_forecast']
        total_revenue += revenue
        total_discount_cost += discount_cost
    else:
        print(f"❌ Do NOT Promote Product: {p}")
        revenue = product_summary.loc[i, 'avg_price'] * product_summary.loc[i, 'weekly_demand_forecast']
        total_revenue += revenue

print("\n--- Summary ---")
print(f"Maximum Expected Revenue: ${total_revenue:,.2f}")
print(f"Total Discount Cost: ${total_discount_cost:,.2f}")
print(f"Budget Utilization: {(total_discount_cost / weekly_discount_budget):.2%}")

Solving the optimization problem...
✅ Solution found.

--- Optimal Promotion Plan ---
✅ Promote Product: 6cdd53843498f92890544667809f1595
✅ Promote Product: 99a4788cb24856965c36a24e339b6058
✅ Promote Product: bb50f2e236e5eea0100680137654686c
✅ Promote Product: d1c427060a0f73f6b889a5c7c61f2ac4
✅ Promote Product: d6160fb7873f184099d9bc95e30376af

--- Summary ---
Maximum Expected Revenue: $2,676.68
Total Discount Cost: $297.41
Budget Utilization: 59.48%
